## **Settings**

- Importing all necessary libraries and modules.

In [1]:
import os, sys, json
from sys import platform
_base_path = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1]) + '/'
sys.path.append(_base_path)
from src.helpers.utils import get_config
from src.modules.explainability import get_explanations

- Definition of paths.

In [2]:
_config = get_config()
json_path = os.path.join(_base_path, _config.get('JSON_FOLDER'))
prompt_path = os.path.join(_base_path, _config.get('PROMPT_FOLDER'))
if platform == 'win32':
	json_path = json_path.replace('/', '\\')
	prompt_path = prompt_path.replace('/', '\\')

- Definition of two different prompts to be tested.

In [4]:
PROMPT = """Using the provided JSON data from a brain MRI study, please generate a comprehensive report detailing the segmentation results of a brain tumor.

Describe the tumor's presence in various brain regions according to the Julich-Brain Atlas, noting the percentages of the tumor within each region and the impact on the region. While discussing the confidence of the segmentation and the model used, keep the explanations high-level and avoid overly technical details.

The goal is to enhance the explainability of the AI segmentation to support clinical decision-making in a manner that can be easily understood by medical professionals.

"""

PROMPT = """Using the provided JSON data from a brain MRI study, please generate a comprehensive report detailing the segmentation results of a brain tumor. The report should be structured as follows:

1. **Summary of Tumor Segmentation**: Provide an overview of the tumor's distribution across various regions of the brain according to the Julich-Brain Atlas. Discuss the semantic segmentation of the tumor into different categories (tumor core, peritumoral edema, GD-enhancing tumor) and their implications.
2. **Detailed Regional Impact**: For each region where the tumor is present, describe the percentage of the tumor within the region and the percentage of the region affected by the tumor. Discuss the known roles and functions of these regions in the brain and how the presence of the tumor might impact these functions.
3. **Semantic Segmentation**: Discuss the different aspects of the tumor identified by the semantic segmentation (tumor core, peritumoral edema, GD-enhancing tumor) and their potential clinical implications.
4. **Clinical Implications**: Discuss how the insights gained from the segmentation results could guide clinical decision-making, including potential treatment strategies and areas for further investigation.

While discussing the confidence of the segmentation and the model used, keep the explanations high-level and avoid overly technical details. The goal is to enhance the explainability of the AI segmentation to support clinical decision-making in a manner that can be easily understood by medical professionals.

"""

- Appending the JSON file to prompt.

In [5]:
with open(os.path.join(json_path, 'SegResNet_sample_2.json'), 'r') as f:
	d = json.load(f)
	PROMPT += str(d)

## **Report generation**

- Testing on **BioMistral** language model.

**NOTES**:

1. For model reference please see the related [model card](https://huggingface.co/BioMistral/BioMistral-7B) on HuggingFace community.

In [9]:
output1 = get_explanations(
	model_id = 'BioMistral/BioMistral-7B',
	prompt = PROMPT,
	write_to_file = True,
	output_path = prompt_path,
	output_length = 1024,
	verbose = True
)

1. 
**Summary of Tumor Segmentation**: The tumor is present in five regions of the brain according to the Julich-Brain Atlas. 
The percentage of the tumor within each region and the percentage of the region affected by the tumor are as follows: 49.22% of the tumor is in the Temporal-to-Parietal (GapMap) right region, with 15.55% of the region affected by the tumor; 14.64% of the tumor is in the Area hOc4la (LOC) right region, with 89.44% of the region affected by the tumor; 12.62% of the tumor is in the Area hOc4lp (LOC) right region, with 84.2% of the region affected by the tumor; 6.36% of the tumor is in the Area hIP4 (IPS) right region, with 97.44% of the region affected by the tumor; and 4.98% of the tumor is in the Area PGp (IPL) right region, with 22.97% of the region affected by the tumor.


- Testing on **Llama-3** language model.

**NOTES**:

1. For model reference please see the related [model card](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) on HuggingFace community.

In [6]:
output2 = get_explanations(
	model_id = 'meta-llama/Meta-Llama-3-8B-Instruct',
	prompt = PROMPT,
	write_to_file = True,
	output_path = prompt_path,
	output_length = 1024,
	verbose = True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

**Summary of Tumor Segmentation**

The brain MRI scan reveals a tumor with a spatial distribution across various regions of the brain, primarily affecting the right hemisphere. 
According to the Julich-Brain Atlas, the tumor is present in regions involved in language processing, memory, and spatial attention. 
The semantic segmentation of the tumor into three categories - tumor core, peritumoral edema, and GD-enhancing tumor - provides valuable insights into the tumor's characteristics and potential clinical implications.

**Detailed Regional Impact**

The tumor is most prominent in the Temporal-to-Parietal region (49.22% of the tumor is located within this region), which is responsible for processing auditory information and language. 
The tumor also affects the Area hOc4la (LOC) right (14.64% of the tumor), Area hOc4lp (LOC) right (12.62% of the tumor), Area hIP4 (IPS) right (6.36% of the tumor), and Area PGp (IPL) right (4.98% of the tumor). 
These regions are involved in various co

- Testing on **Mistral** language model.

**NOTES**:

1. For model reference please see the related [model card](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) on HuggingFace community.

In [5]:
output3 = get_explanations(
	model_id = 'mistralai/Mistral-7B-Instruct-v0.2',
	prompt = PROMPT,
	write_to_file = True,
	output_path = prompt_path,
	output_length = 1024,
	verbose = True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Title: Brain Tumor Segmentation Report using Julich-Brain Atlas

Introduction:
This report provides a comprehensive analysis of a brain MRI scan that detected a tumor, using the provided JSON data. 
The report describes the tumor's presence in various brain regions according to the Julich-Brain Atlas, the impact on each region, and the segmentation results' confidence.

Spatial Distribution of the Tumor:
The tumor was predominantly located in the Temporal-to-Parietal (GapMap) region on the right side, accounting for 49.22% of the tumor. 
This region is part of the temporal lobe and the parietal lobe, playing a crucial role in auditory processing, memory, and spatial awareness. 
The tumor affected 15.55% of this region, potentially leading to symptoms like hearing loss, memory impairment, or spatial disorientation.

Additionally, the tumor infiltrated the Area hOc4la (LOC) on the right side, accounting for 14.64% of the tumor. 
This region is part of the limbic system, responsible for e